This file shall start with an input csv with the schema of full_data_correct_cols_vi.csv

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
version="5"

In [3]:
df = pd.read_csv(f'full_data_correct_cols_v{version}.csv', index_col=0)

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

df.head()

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00,ID,Timeseries ID,Date
0,NaN,0.001986,NaN,0.001986,NaN,0.003970,0.000000,NaN,0.004864,NaN,...,0.003905,NaN,NaN,0.003971,0.000000,NaN,0.001986,Energy_Apiu_1_8_0_BBB6105.kWh,BBB6105,2022-01-01
1,0.353243,0.312500,0.342391,0.320634,0.349195,0.353760,0.372046,0.367966,0.338227,0.314461,...,0.546875,0.542411,0.374942,0.358033,0.462785,0.306167,0.318604,Energy_Apiu_1_8_0_BBB6065.kWh,BBB6065,2022-01-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440781,1.721667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Energy_Ameno_2_8_0_BBB6017.kWh,BBB6017,2022-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333913,0.358832,NaN,...,0.379157,0.363782,NaN,NaN,NaN,NaN,NaN,Energy_Apiu_1_8_0_BBB6017.kWh,BBB6017,2022-01-01
4,1.272458,1.250000,1.261905,1.270764,1.214107,1.292249,1.296913,1.353062,1.334171,1.256536,...,2.250000,2.069509,2.034516,2.078655,1.928294,2.081276,1.304400,Energy_Apiu_1_8_0_BBB6028.kWh,BBB6028,2022-01-01


In [4]:
# Function to replace values within the 21:00:00 to 05:00:00 interval with 0 for IDs containing the term "Ameno" (Only for PV productions)
def replace_night_values_with_zero(df_group):
    no_sunlight_hours = ["20:00:00", "21:00:00", "22:00:00", "23:00:00", "00:00:00", "01:00:00", "02:00:00", "03:00:00", "04:00:00", "05:00:00"]
    for col in df_group.columns[:-3]:
        if ('Ameno' in df_group['ID'].iloc[0]) and ('BBB6052' in df_group['Timeseries ID'].iloc[0] or 'BBB6062' in df_group['Timeseries ID'].iloc[0])  and col in no_sunlight_hours:
            df_group[col].fillna(0, inplace=True)
    return df_group

# Function to interpolate missing values lasting 2 hours or less during the day (07:00-23:00)
def interpolate_short_gaps(df_group):
    for col in df_group.columns[:-3]:
        # Interpolate missing values using linear interpolation along the entire time series
        df_group[col] = df_group[col].interpolate(limit=2, limit_direction='both')
    return df_group

# Function to fill missing values during the night (00:00:00-06:00:00) with the average of the same day of the past 4 weeks for the same ID
def fill_night_gaps(df_group):
    night_hours = ["00:00:00", "01:00:00", "02:00:00", "03:00:00", "04:00:00", "05:00:00", "06:00:00"]
    for col in df_group.columns[:-3]:
        if df_group[col].isnull().all() and col in night_hours:
            # If all values are missing for the column during night hours
            # and there is adjacent data, fill with the average of the same day of the past 4 weeks
            adjacent_data = df_group[col].shift(-1).notnull() | df_group[col].shift(1).notnull()
            if adjacent_data.any():
                end_date = df_group['Date'].iloc[0]
                start_date = end_date - timedelta(weeks=4)
                previous_weeks = df[df['ID'] == df_group['ID'].iloc[0]]
                previous_weeks = previous_weeks[
                    (previous_weeks['Date'].dt.weekday == end_date.weekday()) &
                    (previous_weeks['Date'].dt.time == end_date.time()) &
                    (previous_weeks['Date'] >= start_date) &
                    (previous_weeks['Date'] < end_date)
                ]
                df_group[col].fillna(previous_weeks[col].mean(), inplace=True)

    return df_group

In [5]:
# Apply operations separately for each ID after resetting the index
df_imputed = df.groupby('ID').apply(replace_night_values_with_zero).reset_index(drop=True)
df_imputed = df_imputed.groupby('ID').apply(interpolate_short_gaps).reset_index(drop=True)
df_imputed = df_imputed.groupby('ID').apply(fill_night_gaps).reset_index(drop=True)

# Drop empty DataFrames (corresponding to days removed)
df_imputed = df_imputed.dropna()

In [6]:
df_imputed[df_imputed["Timeseries ID"]=="BBB6052"]

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00,ID,Timeseries ID,Date
2720,0.00,0.0,0.0,0.0,0.0,0.0000,3.905642,12.698714,21.544399,31.162299,...,23.994628,14.441627,5.416202,0.999167,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.Wh,BBB6052,2022-06-30
2721,0.00,0.0,0.0,0.0,0.0,0.0000,3.860530,10.604298,20.021780,28.216426,...,23.811811,14.797325,5.174210,1.104600,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.Wh,BBB6052,2022-07-01
2722,0.00,0.0,0.0,0.0,0.0,0.0000,4.206939,10.916573,21.115469,30.877382,...,24.277444,13.523329,4.514278,0.375000,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.Wh,BBB6052,2022-07-02
2723,0.75,0.0,0.0,0.0,0.0,0.0000,2.751291,10.149034,20.239885,29.554858,...,23.497236,13.288151,5.447399,0.833111,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.Wh,BBB6052,2022-07-03
2724,0.75,0.0,0.0,0.0,0.0,0.0000,3.122557,10.296946,19.648335,29.126942,...,22.892829,13.594143,4.601687,0.867263,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.Wh,BBB6052,2022-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365,0.00,0.0,0.0,0.0,0.0,0.5996,1.643761,5.378791,8.446211,14.184737,...,25.331957,14.463161,6.464116,1.090909,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.kWh,BBB6052,2022-06-25
3366,0.00,0.0,0.0,0.0,0.0,0.0000,3.990646,12.111817,21.460000,30.257409,...,22.764389,13.920706,4.346685,0.967208,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.kWh,BBB6052,2022-06-26
3367,0.00,0.0,0.0,0.0,0.0,0.0000,3.264897,10.992416,15.175821,22.857445,...,0.012690,0.006218,0.001314,0.000500,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.kWh,BBB6052,2022-06-27
3368,0.00,0.0,0.0,0.0,0.0,0.0000,0.002392,0.010102,0.019782,0.030842,...,0.013139,0.010756,0.004000,0.000000,0.0,0.0,0.0,Energy_Ameno_2_8_0_BBB6052.kWh,BBB6052,2022-06-28


In [7]:
df_imputed.to_csv(f"full_data_harmonized_v{version}.csv")